# Support Vector Machine - Home Exercise
Using the Boston Housing Dataset

## Data Preparation

In [ ]:
import pandas as pd
df = pd.read_csv('BostonHousing (1).csv')

# 1. Check for missing values
df.isnull().sum()

In [ ]:
# 2. Check for duplicate rows
df.duplicated().sum()

In [ ]:
# 3. Remove missing values and duplicates
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df.head()

## Simple SVM Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

X = df.drop('medv', axis=1)
y = df['medv']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = SVR(kernel='rbf', C=0.1)
model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae, mse, rmse

In [ ]:
plt.scatter(y_test, predictions)
plt.xlabel('True Prices')
plt.ylabel('Predicted Prices')
plt.title('Simple SVM: True vs Predicted')
plt.grid(True)
plt.show()

## Optimized SVM using GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
    'C': [0.1, 1, 10, 100, 1000]
}

grid = GridSearchCV(SVR(), param_grid, scoring='neg_mean_squared_error', cv=5)
grid.fit(X_train_scaled, y_train)
grid.best_params_

In [ ]:
best_model = grid.best_estimator_
best_preds = best_model.predict(X_test_scaled)

opt_mae = mean_absolute_error(y_test, best_preds)
opt_mse = mean_squared_error(y_test, best_preds)
opt_rmse = np.sqrt(opt_mse)
opt_mae, opt_mse, opt_rmse

In [ ]:
plt.scatter(y_test, best_preds)
plt.xlabel('True Prices')
plt.ylabel('Predicted Prices')
plt.title('Optimized SVM: True vs Predicted')
plt.grid(True)
plt.show()

## Model Deployment

In [ ]:
from sklearn.externals import joblib

scaler_all = StandardScaler()
X_scaled_all = scaler_all.fit_transform(X)

final_model = SVR(kernel=grid.best_params_['kernel'], C=grid.best_params_['C'])
final_model.fit(X_scaled_all, y)

joblib.dump(final_model, 'final_svm_model.joblib')
joblib.dump(scaler_all, 'final_scaler.joblib')